---
# ADAS - Ingest Video and Sensor Streams

To avoid the need for a set of live cameras for this demo, we play back video from a series of JPEG files on disk
and write each video frame to SDP.
These videos are part of the [KITTI Vision Benchmark Suite](http://www.cvlibs.net/datasets/kitti/raw_data.php).

---

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, opencv_image_to_mpl, ImageFileSequenceLoader, UnindexedStream
import os
from pathlib import Path
import pandas as pd
import numpy as np
import base64
from multiprocessing import Process
import json
from itertools import islice
import time

### Define Pravega stream parameters

In [ ]:
gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
video_stream_name = 'adas-raw-video'
sensor_stream_name = 'adas-raw-sensor'

### Initialize connection to Pravega GRPC Gateway - SKIP WHEN RUNNING SIMULATOR

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

In [ ]:
#pravega_client.CreateScope(pravega.pb.CreateScopeRequest(scope=scope))

### Create Pravega stream - SKIP WHEN RUNNING SIMULATOR

In [ ]:
video_stream = UnindexedStream(pravega_client, scope, video_stream_name)
#video_stream.delete_stream()
video_stream.create_stream(min_num_segments=1)
video_stream.truncate_stream()

In [ ]:
sensor_stream = UnindexedStream(pravega_client, scope, sensor_stream_name)
#sensor_stream.delete_stream()
sensor_stream.create_stream(min_num_segments=1)
sensor_stream.truncate_stream()

In [ ]:
# Truncate object detector output stream
detected_stream = OutputStream(pravega_client, scope, 'object-detector-output-video')
#detected_stream.delete_stream()
detected_stream.create_stream(min_num_segments=1)
detected_stream.truncate_stream()

### Ingest JPEG images from files (multiple cameras)

In [ ]:
prefix = '../../data/'
data_dirs = [
    prefix + 'kitti/2011_09_26/2011_09_26_drive_0005_sync',
    prefix + 'kitti/2011_09_26/2011_09_26_drive_0009_sync',
    prefix + 'kitti/2011_09_26/2011_09_26_drive_0011_sync',
    prefix + 'kitti/2011_09_26/2011_09_26_drive_0014_sync',
    prefix + 'kitti/2011_09_26/2011_09_26_drive_0051_sync',
    prefix + 'kitti/2011_09_26/2011_09_26_drive_0059_sync',
    prefix + 'kitti/2011_09_26/2011_09_26_drive_0084_sync',
    prefix + 'kitti/2011_09_29/2011_09_29_drive_0071_sync',
]

In [ ]:
imp.reload(video);
import pravega.video as video
from pravega.video import UnindexedStream, KittiFleetSimulator
imp.reload(video);

In [ ]:
simulator = KittiFleetSimulator(gateway=gateway, scope=scope, video_stream_name=video_stream_name, 
                                sensor_stream_name=sensor_stream_name, data_dirs=data_dirs)

In [ ]:
simulator.run()